In [1]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import time
import socket

In [2]:
class Main:
    def __init__(self, Path = "", Connect = False):
        self.Path = Path
        if len(Path) > 0:
            self.Capture = cv2.VideoCapture(Path)
        else:
            self.Capture = cv2.VideoCapture(0)

        if(Connect == True):
            self.Socket = True
            self.Host, self.Port = "127.0.0.1", 8500
            self.Sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.Sock.connect((self.Host, self.Port))
        else:
            self.Socket = False
        
        self.Food_List = ['fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 
                        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake']
        
        self.Start_Time = 0
        self.Check_Sleep_Timer = 1
        self.Status_Eating = "Eating"
        self.Status_Not_Eating = "Not Eating"
        self.Flag_Eating = self.Status_Not_Eating
    
    def Draw_Boxes(self):
        for Detection in self.Detection_Result.detections:
            Box_Coordinates = Detection.bounding_box
            Start_Point = Box_Coordinates.origin_x, Box_Coordinates.origin_y
            End_Point = Box_Coordinates.origin_x + Box_Coordinates.width, Box_Coordinates.origin_y + Box_Coordinates.height
            cv2.rectangle(self.Frame_Copy, Start_Point, End_Point, (255, 0, 0), 3)

            Category = Detection.categories[0]
            Category_Name = Category.category_name
            Probability = round(Category.score, 2)
            Result_Text = Category_Name + ' (' + str(Probability) + ')'
            Text_Location = (10 + Box_Coordinates.origin_x, 10 + 10 + Box_Coordinates.origin_y)
            cv2.putText(self.Frame_Copy, Result_Text, Text_Location, cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)

        return self.Frame_Copy

    def Classify(self):
        self.Flag_Eating = self.Status_Not_Eating
        self.Base_Options = python.BaseOptions(model_asset_path='efficientdet.tflite')
        self.Options = vision.ObjectDetectorOptions(base_options=self.Base_Options, score_threshold=0.1, max_results=10)
        self.Detector = vision.ObjectDetector.create_from_options(self.Options)

        self.Detection_Result = self.Detector.detect(self.Mp_Image)

        for Detection in self.Detection_Result.detections:
            for Category in Detection.categories:
                if Category.category_name in self.Food_List:
                    self.Flag_Eating = self.Status_Eating
                    break
            if self.Flag_Eating == self.Status_Eating:
                break

    def Process_Frames(self):
        while self.Capture.isOpened():
            self.Ret, self.Frame = self.Capture.read()
            if not self.Ret:
                break
            
            self.Mp_Image = mp.Image(image_format=mp.ImageFormat.SRGB, data=self.Frame)

            if self.Start_Time == 0:
                self.Start_Time = time.time()
            

            if(time.time() - self.Start_Time) >= self.Check_Sleep_Timer:
                self.Classify()
                self.Start_Time = 0
            
            if(self.Socket == True):
                String = f"{self.Flag_Eating}"    
                self.Sock.sendall(String.encode("UTF-8"))
                
            # self.Frame_Copy = np.copy(self.Mp_Image.numpy_view())
            # Labeled_Image = self.Draw_Boxes()
            # self.Frame = cv2.cvtColor(Labeled_Image, cv2.COLOR_BGR2RGB)
                
            cv2.putText(self.Frame, str(self.Flag_Eating), (20, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
            cv2.imshow('Food Detection', self.Frame)

            if (cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Food Detection", cv2.WND_PROP_VISIBLE) < 1):
                break

        self.Capture.release()
        cv2.destroyAllWindows()

In [3]:
Path = "Videoes/Video_1.mp4"
Main = Main(Path)
Main.Process_Frames()